In [1]:
import numpy as np
from TMDP import TMDP
from River_swim import River

from algorithms import *
from model_functions import *

import matplotlib.pyplot as plt


#np.set_printoptions(precision=4)
import math
from utils import *

nS = 50
nA = 2
seed = get_current_seed()
seed = 44697628841978080856580175700798794719
gamma = .9
mu = np.ones(nS) * 1/nS
river = River(nS, mu, small=5, large=1000, seed=seed)
tau = 1.
xi = np.ones(nS) * 1/nS
tmdp = TMDP(river, xi, tau=tau, gamma=gamma, seed=seed)
mdp = TMDP(river, xi, tau=0., gamma=gamma, seed=seed)
tmdp_1 = TMDP(river, xi, tau=.5, gamma=gamma, seed=seed)


Current seed for result reproducibility: 192610091067644126436798985942979815782


In [2]:

res = bellman_optimal_q(tmdp.P_mat_tau, tmdp.env.reward, tmdp.gamma)
Q = res["Q"]
res_1 = bellman_optimal_q(tmdp_1.P_mat_tau, tmdp_1.env.reward, tmdp_1.gamma)
Q_1 = res_1["Q"]
pi = get_policy(Q)
V = compute_V_from_Q(Q, pi)
U = compute_U_from_V(tmdp.env.reward, tmdp.gamma, V)
pi_prime = get_policy(Q_1)
pol_adv_func = compute_policy_advantage_function(Q, V)
print(pol_adv_func)
model_adv_func = compute_model_advantage_function(U, Q)
print(model_adv_func)

[[ 0.00000000e+00 -9.99999915e-02]
 [-8.29576496e-09  0.00000000e+00]
 [-8.26087687e-09  0.00000000e+00]
 [-8.22613488e-09  0.00000000e+00]
 [-8.19153989e-09  0.00000000e+00]
 [-8.15709011e-09  0.00000000e+00]
 [-8.12278467e-09  0.00000000e+00]
 [-8.08862488e-09  0.00000000e+00]
 [-8.05460720e-09  0.00000000e+00]
 [-8.02073341e-09  0.00000000e+00]
 [-7.98700217e-09  0.00000000e+00]
 [-7.95341304e-09  0.00000000e+00]
 [-7.91996468e-09  0.00000000e+00]
 [-7.88665711e-09  0.00000000e+00]
 [-7.85348897e-09  0.00000000e+00]
 [-7.82046117e-09  0.00000000e+00]
 [-7.78757236e-09  0.00000000e+00]
 [-7.75482123e-09  0.00000000e+00]
 [-7.72220776e-09  0.00000000e+00]
 [-7.68973107e-09  0.00000000e+00]
 [-7.65739205e-09  0.00000000e+00]
 [-7.62518937e-09  0.00000000e+00]
 [-7.59312169e-09  0.00000000e+00]
 [-7.56118856e-09  0.00000000e+00]
 [-7.52938956e-09  0.00000000e+00]
 [-7.49772511e-09  0.00000000e+00]
 [-7.46619255e-09  0.00000000e+00]
 [-7.43479278e-09  0.00000000e+00]
 [-7.40352579e-09  0

In [3]:
rel_pol_adv = compute_relative_policy_advantage_function(pi, pi_prime, Q)
print(rel_pol_adv)
rel_model_adv = compute_relative_model_advantage_function(tmdp.env.P_mat, tmdp.xi, U)
print((tmdp.tau - 0.4)*rel_model_adv)

[0.00000000e+00 8.29576496e-09 8.26087687e-09 8.22613488e-09
 8.19153989e-09 8.15709011e-09 8.12278467e-09 8.08862488e-09
 8.05460720e-09 8.02073341e-09 7.98700217e-09 7.95341304e-09
 7.91996468e-09 7.88665711e-09 7.85348897e-09 7.82046117e-09
 7.78757236e-09 7.75482123e-09 7.72220776e-09 7.68973107e-09
 7.65739205e-09 7.62518937e-09 7.59312169e-09 7.56118856e-09
 7.52938956e-09 7.49772511e-09 7.46619255e-09 7.43479278e-09
 7.40352579e-09 7.37238981e-09 7.34138572e-09 7.31051042e-09
 7.27976612e-09 7.24915106e-09 7.21866522e-09 7.18830595e-09
 7.15807591e-09 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
[[ 2.77691989e+00 -1.79280106e-01]
 [-1.63080109e-01 -2.11680099e-01]
 [-2.17080100e-01 -2.17080094e-01]
 [-2.17080095e-01 -2.17080090e-01]
 [-2.17080091e-01 -2.17080085e-01]
 [-2.17080086e-01 -2.17080081e-01]
 [-2.17080082e-01 -2.17080

In [4]:
d = compute_d_from_tau(tmdp.env.mu, tmdp.env.P_mat, tmdp.xi, pi, tmdp.gamma, tmdp.tau)
delta = compute_delta(d, pi)
print(d)
print(delta)

[0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02]
[[0.02 0.  ]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]
 [0.   0.02]]


In [5]:
pol_adv = compute_expected_policy_advantage(rel_pol_adv, d)
model_adv = compute_expected_model_advantage(rel_model_adv, delta)
delta_Q = get_sup_difference_Q(Q)
delta_U = get_sup_difference_U(U)
d_inf_pol = get_d_inf_policy(pi, pi_prime)
d_inf_model = get_d_inf_model(tmdp.env.P_mat, tmdp.xi)
d_exp_pol = get_d_exp_policy(pi, pi_prime, d)
d_exp_model = get_d_exp_model(tmdp.env.P_mat, tmdp.xi, delta)

print("Policy advantage: ",pol_adv)
print("Model advantage: ",model_adv)
print("D_inf_pol: ",d_inf_pol)
print("D_inf_model: ",d_inf_model)
print("D_exp_pol: ",d_exp_pol)
print("D_exp_model: ",d_exp_model)

alpha_tau = compute_alpha_tau(pol_adv, tmdp.gamma, delta_U, d_exp_pol, d_inf_pol)
print("alpha_tau: ",alpha_tau)
alpha_0 = compute_alpha_0(pol_adv, tmdp.tau, tmdp.gamma, delta_U, d_exp_pol, d_inf_pol, d_exp_model, d_inf_model)
print("alpha_0: ",alpha_0)

tau_0 = compute_tau_prime_0(tmdp.tau, model_adv, tmdp.gamma, d_exp_model, delta_U, d_inf_model)
print("tau_prime_0: ", tau_0)

tau_1 = compute_tau_prime_1(tmdp.tau, model_adv, tmdp.gamma, d_exp_model, delta_U, d_inf_model, d_inf_pol, d_exp_pol)
print("tau_prime_1: ", tau_1)



Policy advantage:  5.55360488263546e-09
Model advantage:  5.5541800012625195
D_inf_pol:  2.0
D_inf_model:  1.9600000000000002
D_exp_pol:  1.4400000000000017
D_exp_model:  1.8824000000000034
alpha_tau:  0.0
alpha_0:  -1.14361
tau_prime_0:  0.99976
tau_prime_1:  0.00824


In [6]:
print(delta_U)
print(delta_Q)

1018.0000003235248
20.000000367767676


In [7]:
b_alpha_tau = compute_teleport_bound(alpha_tau, tmdp.tau, tmdp.tau, pol_adv, model_adv,
                                     tmdp.gamma, d_inf_pol, d_inf_model, 
                                     d_exp_pol, d_exp_model, delta_U)
print("b_alpha_tau: ",b_alpha_tau)
b_alpha_0 = compute_teleport_bound(alpha_0, tmdp.tau, 0, pol_adv, model_adv,
                                     tmdp.gamma, d_inf_pol, d_inf_model, 
                                     d_exp_pol, d_exp_model, delta_U)
print("b_alpha_0: ",b_alpha_0)
b_0_tau = compute_teleport_bound(0, tmdp.tau, tau_0, pol_adv, model_adv,
                                     tmdp.gamma, d_inf_pol, d_inf_model, 
                                     d_exp_pol, d_exp_model, delta_U)
print("b_0_tau: ",b_0_tau)
b_1_tau = compute_teleport_bound(1, tmdp.tau, tau_1, pol_adv, model_adv,
                                     tmdp.gamma, d_inf_pol, d_inf_model, 
                                     d_exp_pol, d_exp_model, delta_U)
print("b_1_tau: ",b_1_tau)

b_alpha_tau:  -16.941600000000033
b_alpha_0:  20471.873810280096
b_0_tau:  -16.93703176667975
b_1_tau:  -580785.8309551951


In [8]:
tb_alpha_tau = compute_teleport_bound_alpha_tau(tmdp.tau, pol_adv, gamma, d_inf_pol, d_exp_pol, d_exp_model, delta_U)
print("tb_alpha_tau: ",tb_alpha_tau)
tb_alpha_0 = compute_teleport_bound_alpha_0(tmdp.tau, pol_adv, model_adv, gamma, d_inf_pol, d_exp_pol, d_inf_model, d_exp_model, delta_U)
print("tb_alpha_0: ",tb_alpha_0)

tb_0_tau = compute_teleport_bound_0_tau(tmdp.tau, model_adv, gamma, d_inf_model, d_exp_model, delta_U)
print("tb_0_tau: ",tb_0_tau)

tb_1_tau = compute_teleport_bound_1_tau(tmdp.tau, pol_adv, model_adv, gamma, d_inf_model, d_inf_pol, d_exp_model, d_exp_pol, delta_U)
print("tb_1_tau: ",tb_1_tau)

tb_alpha_tau:  -16.941600000000033
tb_alpha_0:  20471.873810442947
tb_0_tau:  -33.87456531824877
tb_1_tau:  17650.12663225304


In [9]:
tb_alpha_0 = compute_teleport_bound_alpha_0_test(tmdp.tau, pol_adv, model_adv, gamma, d_inf_pol, d_exp_pol, d_inf_model, d_exp_model, delta_U)
print("tb_alpha_0: ",tb_alpha_0)

tb_1_tau = compute_teleport_bound_1_tau_test(tmdp.tau, pol_adv, model_adv, gamma, d_inf_model, d_inf_pol, d_exp_model, d_exp_pol, delta_U)
print("tb_1_tau: ",tb_1_tau)

tb_alpha_0:  20471.87381044294
tb_1_tau:  17616.52465489762


In [10]:
q_c = np.zeros((nS, nA))
tmdp.update_tau(.5)
cur_res = curriculum_MPI(tmdp, q_c, alpha = 0.25, alpha_pol = 0.2, episodes=1000000, batch_nS=1, temperature=15., biased=False, lam=0.)


[(0.0, 0.4996), (1.0, 0.49968)]
[3.457099321252606e-05, 6.903338546064173e-05]
Updating the policy with alpha_star:  1 tau_star:  0.49968
[(0.0, 0.49928), (1.0, 0.49928)]
[3.450097816314857e-05, 3.450097816314857e-05]
Updating the policy with alpha_star:  1 tau_star:  0.49928
[(0.0, 0.49888), (1.0, 0.49889)]
[3.441041256960103e-05, 3.320952263161437e-05]
Updating the policy with alpha_star:  0.0 tau_star:  0.49888
[(0.0, 0.49848), (1.0, 0.49849)]
[3.432434395597784e-05, 3.312560647298852e-05]
Updating the policy with alpha_star:  0.0 tau_star:  0.49848
[(0.0, 0.49808), (1.0, 0.49809)]
[3.434586556548387e-05, 3.314659077697811e-05]
Updating the policy with alpha_star:  0.0 tau_star:  0.49808
[(0.0, 0.49768), (1.0, 0.49771)]
[3.4342022197235746e-05, 3.166722581764818e-05]
Updating the policy with alpha_star:  0.0 tau_star:  0.49768
[(0.0, 0.49728), (1.0, 0.49731)]
[3.436359513499266e-05, 3.168718138480166e-05]
Updating the policy with alpha_star:  0.0 tau_star:  0.49728
[(0.0, 0.49688), 

In [13]:
res = bellman_optimal_q(mdp.env.P_mat, mdp.env.reward, mdp.gamma)
Q = res["Q"]

d = compute_d_from_tau(mdp.env.mu, mdp.env.P_mat, mdp.xi, get_policy(Q), mdp.gamma, 0.)
d_curr = compute_d_from_tau(tmdp.env.mu, tmdp.P_mat_tau, tmdp.xi, get_policy(get_softmax_policy(cur_res["thetas"][-1])), tmdp.gamma, 0.)

print(get_policy(Q) - get_policy(get_softmax_policy(cur_res["thetas"][-1])))

r_s_a = compute_r_s_a(mdp.env.P_mat, mdp.env.reward)

j_opt = compute_j(r_s_a, get_policy(Q), d, mdp.gamma)
j_curr = compute_j(r_s_a, get_softmax_policy(cur_res["Qs"][-1]), d_curr, tmdp.gamma)
print("optimal performance: ",j_opt, "curriculum performance: ",j_curr)

[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [-1.  1.]
 [-1.  1.]
 [-1.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]
optimal performance:  70.66229588024339 curriculum performance:  70.50796068609186
